In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("akila").getOrCreate()
emp_data =[
["001", "101", "John Doe", "30", "Male", "50000", "2015-01-01"],
["002","101", "Jane Smith", "25", "Female", "45000", "2016-02-15"],
["003", "102", "Bob Brown", "35", "Male", "55000", "2014-05-01"],
["004","102", "Alice Lee","28", "Female", "48000", "2017-09-30"],
["005", "103","Jack Chan", "40", "Male", "60000", "2013-04-01"],
["006","103","Jill Wong", "32", "Female", "52000", "2018-07-01"],
["007","101","James Johnson", "42", "Male", "70000", "2012-03-15"],
["008","102", "Kate Kim", "29", "Female", "51000", "2019-10-01"],
["009","103", "Tom Tan","33", "Male", "58000", "2016-06-01"],
["010","104", "Lisa Lee", "27", "Female", "47000", "2018-08-01"],
["011", "104", "David Park","38", "Male", "65000", "2015-11-01"],
["012","105", "Susan Chen","31", "Female", "54000", "2017-02-15"],
["013", "106", "Brian Kim", "45", "Male", "75000", "2011-07-01"],
["014", "107", "Emily Lee", "26", "Female", "46000", "2019-01-01"],
["015","106", "Michael Lee", "37", "Male", "63000","2014-09-30"],
["016", "107", "Kelly Zhang", "30", "Female", "49000", "2018-04-01"],
["017","105", "George Wang", "34", "Male", "57000", "2016-03-15"],
["018","104", "Nancy Liu", "29", "Female", "50000", "2017-06-01"],
["019","103", "Steven Chen", "36", "Male", "62000", "2015-08-01"],
["020","102", "Grace Kim", "32", "Female", "53000", "2018-11-01"]
]
emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

spark.createDataFrame(data=emp_data, schema=emp_schema).write.mode("overwrite").saveAsTable("employee")

df = spark.createDataFrame(data=emp_data, schema=emp_schema)

display(df)

In [0]:
spark.createDataFrame(data=emp_data, schema=emp_schema).write.mode("overwrite").saveAsTable("employee")

In [0]:
df = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

window_Spec = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())
maxfunc = max(col("salary")).over(window_Spec)
dfd = df.withColumn("maxSalary", maxfunc)

display(dfd)

In [0]:
windowSpec1 = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())

rnOverClause = row_number().over(windowSpec1)

dfrns = df.withColumn("RowNumberOnSalary", rnOverClause)

In [0]:
dfrns.filter(col("RowNumberOnSalary") == 3).show()

In [0]:
dfdndr = df.withColumn("denseRank" , expr("dense_rank() over(partition By department_id order by salary desc)"))

display(dfdndr)


In [0]:
df.repartition(3).write.mode("overwrite").format("delta").save("/delta/df3")

In [0]:
df.repartition(9).write.mode("overwrite").format("csv").save("data1/df")

In [0]:
dbutils.fs.ls("data1/df/")

In [0]:
dbutils.fs.ls("/delta/df3")


In [0]:
from pyspark.sql.functions import *


df.withColumn("partitionNum" , spark_partition_id()).repartition(6,"department_id").write.mode("overwrite").save("/data/dfg/")

In [0]:
dbutils.fs.ls("/data/dfg/")

In [0]:
import pandas as pd
import random
from faker import Faker


# Initialize Faker
fake = Faker()

# ----- Department DataFrame -----
departments = [
    {"DepartmentID": 1, "DepartmentName": "Human Resources"},
    {"DepartmentID": 2, "DepartmentName": "Finance"},
    {"DepartmentID": 3, "DepartmentName": "Engineering"},
    {"DepartmentID": 4, "DepartmentName": "Marketing"},
    {"DepartmentID": 5, "DepartmentName": "Sales"}
]

df_departments = pd.DataFrame(departments)

# ----- Employee DataFrame -----
employees = []

for emp_id in range(1, 101):
    employees.append({
        "EmployeeID": emp_id,
        "FirstName": fake.first_name(),
        "LastName": fake.last_name(),
        "Email": fake.email(),
        "HireDate": fake.date_between(start_date='-10y', end_date='today'),
        "Salary": round(random.uniform(40000, 120000), 2),
        "DepartmentID": random.choice(df_departments["DepartmentID"].tolist())
    })

df_employees = pd.DataFrame(employees)
# Display the first few rows
print("Departments:")
print(df_departments)
print("\nEmployees:")
print(df_employees.head())


In [0]:
df_employees = pd.DataFrame(employees)


In [0]:
df_employees_pys = spark.createDataFrame(df_employees)


In [0]:
display(df_employees_pys)

In [0]:
display(df_departments)

In [0]:
dfpysDepartment = spark.createDataFrame(df_departments)

In [0]:
dfInnerJoin = df_employees_pys.join(dfpysDepartment, on='DepartmentID', how='inner')
display(dfInnerJoin)

In [0]:
leftDf = df_employees_pys.join(dfpysDepartment, df_employees_pys.DepartmentID == dfpysDepartment.DepartmentID , "left")

In [0]:
display(leftDf)

In [0]:
DFfINAL = df_employees_pys.join(dfpysDepartment, "left_outer",(df_employees_pys.DepartmentID == dfpysDepartment.DepartmentID) & ((df_employees_pys.EmployeeID == 10) |(df_employees_pys.EmployeeID == 20)|(df_employees_pys.EmployeeID == 65)|(df_employees_pys.EmployeeID == 35)) & (df_employees_pys.Salary.isNotNull()))
display(DFfINAL)